# Python для анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

### Работа с API: пример с API ВКонтакте

Сегодня мы немного поработаем с API. API – программный интерфейс приложения, сокращение от *Application Programming Interface*. Этот интерфейс позволяет выполнять различные операции автоматически, через приложение. Если API нам нужен исключительно как источник данных, можно писать запросы, позволяющие обратиться к хранилищу информации внутри API. Если мы хотим управлять приложением, которое будет выполнять какие-то действия, удаленно, можно написать код, который будет, например, автоматически отвечать на сообщения, когда мы не онлайн, лайкать новый пост друга через 30 секунд после его появления, пересылать на почту фотографии, которые выложили участники диалога и прочее.

Мы будем работать с API социальной сети ВКонтакте. Использовать API для написания и приема сообщений средствами Python мы не будем, а рассмотрим API как источник данных, позволяющий выгрузить все посты со стены пользователя или сообщества. Для работы нам понадобится библиотека `vk`, ее нужно установить через `pip install vk` в *Anaconda Command Prompt* (см. пример с `selenium` [здесь](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/11-06/lect-selenium1.ipynb)).

Импортируем библиотеку:

In [1]:
import vk

Теперь нужно авторизоваться: создать приложение и получить токен доступа к нему.

#### Для авторизации:
    
1. [Создать](https://vk.com/apps?act=manage) приложение ВКонтакте (пройти по ссылке). Дать название, выбрать *Standalone-приложение*.
2. Включить приложение, сделать публичным (*Настройки - Состояние* и выбрать в выпадающем меню *Приложение включено и видно всем*).
3. Авторизоваться: скопировать строку ниже в браузер, в `client id` вместо 1 выставить `id` своего приложения (первая строка в настроках ‒ *ID приложения*). Если не хочется ни в чем ограничивать свое приложение, можно оставить `scope=all` (у приложения будет доступ ко всему, к чему есть доступ у пользователя).

    > ```https://oauth.vk.com/authorize?client_id=1&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token```

4. Скопировать `access token` из обновленной адресной строки (все после `access_token=` и до `&expires_in`, без `&`). Никому не показывать! По этому токену можно получить доступ ко всему аккаунту.

In [2]:
# скопировать свой access token без пробелов вместо 1234
token = '1234'

In [3]:
session = vk.Session(access_token = token) # открыть сессию для работы
api = vk.API(session) # подключиться к API

Попробуем сгрузить первые 100 постов со [стены](https://vk.com/hseofficial) неофициального сообщества ВШЭ. Сохраним ссылку на сообщество в переменную `group`:

In [4]:
group = 'hseofficial'

Обратите внимание: ссылка должна быть относительной, то есть без части с `https:/vk.com/`. Python и так будет знать, что мы работаем с сетью ВКонтакте.

Теперь получим доступ к стене этого сообщества:

In [5]:
# wall.get - "подключаемся" к стене
# count - сколько постов выгрузить (максимум)
# v - версия API, можно обойтись без нее, но Python может выдать warning

res = api.wall.get(domain = group, count = 100, v = 5.73)

In [7]:
res

Результат, сохраненный в `res`, очень похож на словарь. На самом деле, многие API возвращают результаты в формате JSON (JavaScript Object Notation), который тоже может быть представлен в виде набора пар ключ-значение.

In [8]:
res.keys()

dict_keys(['count', 'items'])

Ключами являются `count` и `items`. Нужные нам объекты (текст постов, id автора, дата и время публикации и проч.) находятся в `items`.

In [9]:
res['items'][0] # первый элемент items - первый пост со всей информацией о нем

{'id': 32590,
 'from_id': -132,
 'owner_id': -132,
 'date': 1549025481,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'copy_history': [{'id': 340,
   'owner_id': -130374393,
   'from_id': -130374393,
   'date': 1548750547,
   'post_type': 'post',
   'text': 'Итак! После каникул оркестр НИУ ВШЭ опять в строю! \n \nС приходом нового года мы хотим найти новые лица в наш коллектив! \n \nЕсли ты играешь на музыкальном инструменте и любишь искусство так же, как и мы. Добро пожаловать! \n \nКогда: \nПн, пт - с 18:30 до 21:00 \n \nГде: \nул. Старая Басманная 21/4 \n \nЧто потребуется: \n• Музыкальный инструмент \n• Хорошее настроение \n• Уверенность в себе \n \nМы ждём именно тебя! \n \nПо всем вопросам писать или звонить [id133931431|Поле]: \n89262337123',
   'attachments': [{'type': 'photo',
     'photo': {'id': 456239403,
      'album_id': -7,
      'owner_id': -130374393,
      'user_id': 100,
      'photo_75': 'https://pp.userapi.com/c848524/v848524534/117299/b54BUX2-nbk.jpg',


Помимо текста поста можно найти много всего интересного. Например, тип поста (`post_type`), дата (`date`), id поста (`id`), лайки (`likes`, которые включают информацию о том, могут ли пользователи лайкать пост и публиковать его, а также собственно число лайков), репосты (`reposts`, которые включают число репостов), число просмотров (`views`), комментарии (`comments`, которые включают информацию о том, могут ли пользователи комментировать пост, и число комментариев), и так далее.

Давайте остановимся на тексте поста, id автора, id поста, дате публикации, числе лайков и репостов. Чтобы извлечь соответствующую информацию, сохраним `items` и извлечем из них нужные поля:

In [10]:
items = res['items']

In [11]:
full_list = []

for item in items:
    L = [item['from_id'], item['id'], item['text'], item['date'],
        item['likes']['count'], item['reposts']['count']]  # нужные поля
    full_list.append(L)  # добавляем в список списков full_list

In [12]:
# несколько элементов списка
full_list[0:3]

[[-132, 32590, '', 1549025481, 2, 0],
 [-132,
  32586,
  'Поздравляем Школу логистики ФБМ с высокой оценкой образовательной программы "Стратегическое управление логистикой" за рубежом! https://www.hse.ru/news/edu/235206567.html',
  1548532201,
  5,
  2],
 [-132, 32585, '', 1548146499, 6, 0]]

Видно, что в некоторых постах текста не обнаружено. 

Из этого списка списков можно легко сделать датафрейм `pandas`. Но прежде посмотрим, как сгрузить следующие 100 (и не только 100) постов со стены. Обычно при работе с API нужно принимать во внимание две вещи: 1) какое ограничение стоит на число запросов за один раз (число постов в нашем случае); 2) какое ограничение стоит на число запросов в минуту. Чтобы действовать в соответствии с этими ограничениями, поступим так: будем грузить каждые следующие 100 постов, добавлять их к нашему списку, потом немного ждать, и грузить еще 100 постов, и так до тех пор, пока не сгрузим желаемое количество.

Давайте для начала выберем это желаемое число постов. Пусть будет 400. 

In [13]:
nposts = 400

Теперь вопрос: по каким значениям нужно «пробегаться» в цикле, чтобы сгрузить посты с 100 по 400 (первые 100 уже сохранены в `res`)? По целым значениям от 2 до 4 включительно, умножая эти значения на 100. В `vk.get` есть опция `offset`. Она позволяет сдвинуть начало отсчета постов на некоторое число. Так, если мы напишем `api.wall.get(domain = group, count = 100, offset = 100, v = 5.73)`,  мы получим посты с 100 по 200, так как начало отсчета сдвинулось на 100.

Реализуем описанное выше. Для цикла нам понадобится `range()`, а для задержки после выгрузки каждой сотни постов нам пригодится функция `sleep`: 

In [14]:
from time import sleep

In [15]:
for i in range(2, int(nposts/100) + 1):
    res2 = api.wall.get(domain = group, count = 100, offset = 100 * i, v = 5.73)
    items2 = res2['items']
    items.extend(items2) # добавляем к первой сотне постов в items
    sleep(10)

In [16]:
# опять выберем только нужные поля
full_list = []
for item in items:
    L = [item['from_id'], item['id'], item['text'], item['date'],
        item['likes']['count'], item['reposts']['count']]
    full_list.append(L)

Оставлось превратить обновленный список `items` (список списков) в датафрейм. Импортируем `pandas`.

In [17]:
import pandas as pd

Создадим датафрейм:

In [18]:
df = pd.DataFrame(full_list)

In [19]:
df.head(10)

,0,1,2,3,4,5
0,-132,32590,,1549025481,2,0
1,-132,32586,Поздравляем Школу логистики ФБМ с высокой оцен...,1548532201,5,2
2,-132,32585,,1548146499,6,0
3,-132,32580,Проблемы качества питания в студенческих столо...,1547797174,10,0
4,-132,32578,Ректор НИУ ВШЭ определил ориентиры для абитури...,1547399794,31,5
5,-132,32577,Дорогие наши студенты! Берегите пропуска и соц...,1547319555,2,0
6,-132,32576,"Данилова Александра Александровна, найдена кар...",1547319476,0,0
7,-132,32575,На Полежаевской карта у кассира,1547319279,2,1
8,-132,32574,"Всем известно, что Лицей НИУ ВШЭ создан по абс...",1547319023,15,3
9,-132,32543,"Веб-журнал ""Популярная логистика"" https://vk.c...",1544212600,8,3


Ура! Осталось только дать внятные названия столбцам и разобраться, почему дата представлена в таком виде. что делать со столбцами, мы уже знаем.

In [20]:
df.columns = ['from_id', 'id', 'text', 'date', 'likes', 'reposts']
df.head(10)

,from_id,id,text,date,likes,reposts
0,-132,32590,,1549025481,2,0
1,-132,32586,Поздравляем Школу логистики ФБМ с высокой оцен...,1548532201,5,2
2,-132,32585,,1548146499,6,0
3,-132,32580,Проблемы качества питания в студенческих столо...,1547797174,10,0
4,-132,32578,Ректор НИУ ВШЭ определил ориентиры для абитури...,1547399794,31,5
5,-132,32577,Дорогие наши студенты! Берегите пропуска и соц...,1547319555,2,0
6,-132,32576,"Данилова Александра Александровна, найдена кар...",1547319476,0,0
7,-132,32575,На Полежаевской карта у кассира,1547319279,2,1
8,-132,32574,"Всем известно, что Лицей НИУ ВШЭ создан по абс...",1547319023,15,3
9,-132,32543,"Веб-журнал ""Популярная логистика"" https://vk.c...",1544212600,8,3


С датой все интереснее. То, что указано в столбце `date`, это дата в виде UNIX-времени (POSIX-времени). Это число секунд, прошедших с 1 января 1970 года. Несмотря на то, что такой формат даты-времени кажется необычным, он довольно широко распространен в разных системах и приложениях (см. подробнее [здесь](https://ru.wikipedia.org/wiki/Unix-%D0%B2%D1%80%D0%B5%D0%BC%D1%8F)). Этот факт, конечно, радует, но хочется получить дату в более человеческом формате. Давайте напишем функцию для перевода UNIX-времени в формат *год-месяц-день-часы-минуты-секунды*. Для этого нам понадобится модуль *datetime*.

In [21]:
from datetime import datetime

In [22]:
def date_norm(date):
    d = datetime.fromtimestamp(date) # timestamp - UNIX-время в виде строки
    str_d = d.strftime("%Y-%m-%d %H:%M:%S") # %Y-%m-%d %H:%M:%S - год-месяц-день, часы:мин:сек
    date_norm, time_norm = str_d.split(' ') # разобьем результат на части, отделим дату от времени
    return date_norm, time_norm

Применим нашу функцию к элементам столбца *date* и создадим новый – *date_norm*.

In [23]:
df['date_norm'] = df.date.apply(date_norm)

In [24]:
df.head()

,from_id,id,text,date,likes,reposts,date_norm
0,-132,32590,,1549025481,2,0,"(2019-02-01, 15:51:21)"
1,-132,32586,Поздравляем Школу логистики ФБМ с высокой оцен...,1548532201,5,2,"(2019-01-26, 22:50:01)"
2,-132,32585,,1548146499,6,0,"(2019-01-22, 11:41:39)"
3,-132,32580,Проблемы качества питания в студенческих столо...,1547797174,10,0,"(2019-01-18, 10:39:34)"
4,-132,32578,Ректор НИУ ВШЭ определил ориентиры для абитури...,1547399794,31,5,"(2019-01-13, 20:16:34)"


Можно было, конечно, не разбивать на части дату и время, сохранять одной строкой. А можно написать функции, которые будут отделять дату от времени – извлекать их из кортежа в `date_norm`.

In [25]:
def get_date(date):
    return date[0]

def get_time(date):
    return date[1]

In [26]:
df['Date'] = df.date_norm.apply(get_date)
df['Time'] = df.date_norm.apply(get_time)

In [27]:
df.head()

,from_id,id,text,date,likes,reposts,date_norm,Date,Time
0,-132,32590,,1549025481,2,0,"(2019-02-01, 15:51:21)",2019-02-01,15:51:21
1,-132,32586,Поздравляем Школу логистики ФБМ с высокой оцен...,1548532201,5,2,"(2019-01-26, 22:50:01)",2019-01-26,22:50:01
2,-132,32585,,1548146499,6,0,"(2019-01-22, 11:41:39)",2019-01-22,11:41:39
3,-132,32580,Проблемы качества питания в студенческих столо...,1547797174,10,0,"(2019-01-18, 10:39:34)",2019-01-18,10:39:34
4,-132,32578,Ректор НИУ ВШЭ определил ориентиры для абитури...,1547399794,31,5,"(2019-01-13, 20:16:34)",2019-01-13,20:16:34


Всё! Материалы о разных методах и функциях для `vk.api` можно найти в [официальной документации](https://vk.com/dev/manuals). 